In [1]:
%matplotlib inline



import pandas as pd

df = pd.read_csv("./small_set/TTS1.csv", header=None , encoding="big5")



- TT	TT1	郵件狀態代碼	X(2) | Status_code
- TT	TT2	掛號號碼	X(20) | Mail_num
- TT	TT3	處理日期	X(10) | Mail_date
- TT	TT4	處理時間	X(8) | Mail_time
- TT	TT5	處理局號	X(6) | OP_office // operation office
- TT	TT6	其它	X(42) | other


In [2]:
df.columns = ["Status_code", "Mail_num", "Mail_date", 
             "Mail_time", "OP_office", "other"]

In [3]:
df.head(20)

,Status_code,Mail_num,Mail_date,Mail_time,OP_office,other
0,Y4,00000000000000,2018-01-01,09:49:04,330031,003
1,Y4,00000000000000,2018-01-01,09:58:08,330031,002
2,I4,00000000000000,2018-01-01,14:11:51,330031,
3,Y4,00000000000000,2018-01-01,14:23:32,330031,006
4,I4,00000000000000,2018-01-01,14:52:09,330031,
5,I4,00000000000000,2018-01-01,16:15:52,330031,
6,Y4,58668700100170,2018-01-01,11:06:44,100250,001
7,I3,59928400100170,2018-01-01,21:10:45,220014,
8,P5,75233300100170,2018-01-01,05:30:09,704583,
9,A3,19491400101070,2018-01-01,21:40:46,320008,65


# 定義：狀態碼為 **點**

In [30]:
df['OP_office'] = df['OP_office'].astype(str)


OPnodes = list(df.OP_office.unique())

head_OPnodes =sorted(list(set([node[0:3] for node in OPnodes])))
head_OPnodes
dic_head_OPnodes  = dict( zip(head_OPnodes, range(len(head_OPnodes))) )
dic_head_OPnodes

label_OPnodes = lambda x: dic_head_OPnodes[x[0:3]]

label_OPnodes("330031")

data_OPnodes = []
for node in OPnodes:
    data = { "id": node, "group": label_OPnodes(node)}
    data_OPnodes.append(data)
data_OPnodes

{'100': 0,
 '200': 1,
 '220': 2,
 '241': 3,
 '260': 4,
 '300': 5,
 '320': 6,
 '330': 7,
 '360': 8,
 '400': 9,
 '420': 10,
 '500': 11,
 '540': 12,
 '600': 13,
 '640': 14,
 '704': 15,
 '730': 16,
 '800': 17,
 '830': 18,
 '880': 19,
 '900': 20,
 '918': 21,
 '950': 22,
 '970': 23}

In [79]:
nodes = list(df.Status_code.unique())

nodes

head_nodes = sorted(list(set([ node[0] for node in nodes])))

head_nodes

dic_nodes  = dict( zip(head_nodes, range(len(head_nodes))) )

dic_nodes

label_node = lambda x: dic_nodes[x[0]]

label_node("T4")

data_nodes = []
for node in nodes:
    data = { "id": node, "group": label_node(node)}
    data_nodes.append(data)

list

# 定義：**線**，狀態的改變
- 要依據狀態、郵件號碼及時間去決定**線** 的連接
- 時間的轉換，請參考 [pandas.to_datetime()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

In [48]:
df['Mail_datetime'] = df.Mail_date+" "+df.Mail_time

In [49]:
df.Mail_datetime = pd.to_datetime(df.Mail_datetime)

```
all_mail["58668700100170"] = [ ("Y4", "2018-01-01 09:49:04"),
                           ("I4", "2018-01-01 14:11:51"), ... ] 

```

In [50]:
all_mail = {}
all_mail_key = set()
for idx, row in df.iterrows():
    mail_code = row.Mail_num.strip()
    if not mail_code in all_mail_key:
        all_mail[mail_code] = []
        all_mail_key.add(mail_code)
        
    all_mail[mail_code].append( (row.Status_code, row.Mail_datetime, row.OP_office) )

In [52]:
len(all_mail.keys())

5982

In [62]:
all_mail['58668700100170'][0][2]

'100250'

In [76]:
#def convert_2_edge(mail_status):
#    edges = []
#    for idx in range(len(mail_status)-1):
#        edges.append( ( mail_status[idx][0], mail_status[idx+1][0]) )
        
#    return edges
# "Y4" -> "H4", 
# "H4" -> "Z2"

#mail_status = all_mail['58668700100170']
#convert_2_edge(mail_status)

[('Y4', 'I4')]

In [65]:
def convert_2_edge(mail_status):
    edges = []
    for idx in range(len(mail_status)-1):
        edges.append( ( mail_status[idx][2], mail_status[idx][0],mail_status[idx+1][2]) )
        
    return edges
# "Y4" -> "H4", 
# "H4" -> "Z2"

mail_status = all_mail['58668700100170']
convert_2_edge(mail_status)

[('100250', 'Y4', '100250')]

In [67]:
all_edges = [] 
for mail_code in all_mail:
    status_num = len(all_mail[mail_code])
    
    if (status_num) > 1:
        mail_status = all_mail[mail_code]
        all_edges.extend(convert_2_edge(mail_status))
all_edges

[('330031', 'Y4', '330031'),
 ('330031', 'Y4', '330031'),
 ('330031', 'I4', '330031'),
 ('330031', 'Y4', '330031'),
 ('330031', 'I4', '330031'),
 ('330031', 'I4', '900030'),
 ('900030', 'Y4', '500055'),
 ('500055', 'Y4', '100029'),
 ('100029', 'G2', '900030'),
 ('900030', 'I4', '500055'),
 ('500055', 'I4', '330031'),
 ('330031', 'Y4', '330031'),
 ('100250', 'Y4', '100250'),
 ('704583', 'P5', '704583'),
 ('420009', 'Y4', '420009'),
 ('420009', 'H4', '420009'),
 ('500027', 'H4', '500027'),
 ('500600', 'Z2', '500600'),
 ('330585', 'Y4', '330585'),
 ('640582', 'W2', '640582'),
 ('100603', 'W2', '100603'),
 ('500027', 'H4', '500027'),
 ('500027', 'H4', '500027'),
 ('640582', 'W2', '640582'),
 ('100189', 'Y4', '100189'),
 ('100189', 'Y4', '100189'),
 ('500027', 'H4', '500027'),
 ('800586', 'P5', '800586'),
 ('640582', 'W2', '640582'),
 ('500027', 'H4', '500027'),
 ('540582', 'W2', '540582'),
 ('100601', 'W2', '100601'),
 ('500058', 'Y4', '500058'),
 ('100603', 'W2', '100603'),
 ('500027', 'H

In [69]:
import collections 
import numpy as np

In [70]:
value_list = {}
for edge, value in collections.Counter(all_edges).most_common(1000):
    value_list[edge] = value

list_max = np.max(list(value_list.values()))
list_min = np.min(list(value_list.values()))
list_diff = float(list_max - list_min)

k = 20

normal_val = lambda x: int(k/3+k*2*(x-list_min)/list_diff)

In [73]:
normal_val(1700)

380

In [83]:
data_edges = []
for edge, value in collections.Counter(all_edges).most_common(1000):
    edge = {"source": edge[0], "stage":edge[1],"target1":edge[2], "value":normal_val(value)}
    data_edges.append(edge)
data_edges

[{'source': '400003', 'stage': 'A1', 'target1': '400672', 'value': 46},
 {'source': '704586', 'stage': 'Y4', 'target1': '704586', 'value': 44},
 {'source': '300584', 'stage': 'Y4', 'target1': '300584', 'value': 28},
 {'source': '100012', 'stage': 'A1', 'target1': '91813', 'value': 27},
 {'source': '400008', 'stage': 'A1', 'target1': '400672', 'value': 24},
 {'source': '400006', 'stage': 'A1', 'target1': '400672', 'value': 23},
 {'source': '704585', 'stage': 'Y4', 'target1': '704585', 'value': 22},
 {'source': '830000', 'stage': 'A1', 'target1': '800672', 'value': 22},
 {'source': '704584', 'stage': 'Y4', 'target1': '704584', 'value': 21},
 {'source': '100572', 'stage': 'Y4', 'target1': '100572', 'value': 21},
 {'source': '100581', 'stage': 'Y4', 'target1': '100581', 'value': 21},
 {'source': '100591', 'stage': 'Y4', 'target1': '100591', 'value': 21},
 {'source': '220600', 'stage': 'Y4', 'target1': '220600', 'value': 19},
 {'source': '100587', 'stage': 'Y4', 'target1': '100587', 'value'

In [80]:
all_data = { "nodes":data_OPnodes, "links":data_edges}

In [81]:
import json

In [82]:
open("OPdata_4_d3.json", 'w').write(json.dumps(all_data, indent=2))

90880

## test on op_office

In [26]:
mail_op = {}
for idx, row in df.head(1000).iterrows():
    m_code = row.Mail_num.strip()
    if not m_code in mail_op:
        mail_op[m_code] = []
    mail_op[m_code].append( (row.Status_code, row.OP_office, row.Mail_datetime))

In [27]:
def OP_seq(mail_seq):
    seq = []
    for idx in range(len(mail_seq)-1):
        seq.append( ("%s-%s"%(mail_seq[idx][0], mail_seq[idx+1][0]), mail_seq[idx][1], mail_seq[idx+1][1]))
    return seq

In [28]:
mail_op_seq = []
for code in mail_op:
    if (len(mail_op[code])) > 1:
        mail_op_seq.append(OP_seq(mail_op[code]))

In [29]:
mail_op_seq

[[('Y4-I4', 100601, 100601)],
 [('H4-W2', 500027, 500027)],
 [('Y4-H4', 100564, 100564)],
 [('W2-I4', 640582, 640582)],
 [('Y4-I4', 330585, 330585)],
 [('Y4-H4', 420009, 420009), ('H4-H4', 420009, 420009)],
 [('H4-W2', 500027, 500027)],
 [('Z2-P4', 500600, 500600)],
 [('H4-W2', 500027, 500027)],
 [('Y4-I4', 100189, 100189)],
 [('Z4-Y4', 91814, 100587), ('Y4-I4', 100587, 100587)],
 [('I4-I4', 260000, 260000)],
 [('Y4-I4', 800586, 800586)],
 [('V2-Z4', 330585, 330585)],
 [('Y4-I4', 800586, 800586)],
 [('W2-I4', 640582, 640582)],
 [('H4-W2', 500027, 500027)],
 [('Y4-H4', 500584, 500584), ('H4-Z2', 500584, 500584)],
 [('H4-W2', 500027, 500027)],
 [('Y4-I4', 360023, 360023)],
 [('W2-I4', 640582, 640582)],
 [('Y4-I4', 540028, 540028)],
 [('Y4-I4', 900043, 900043)],
 [('H4-W2', 500027, 500027)],
 [('H4-W2', 500027, 500027)],
 [('H4-W2', 500027, 500027)],
 [('H4-W2', 500027, 500027)],
 [('I4-I4', 320008, 320008)],
 [('Y4-Y4', 540028, 540028),
  ('Y4-I4', 540028, 540028),
  ('I4-I4', 540028, 54